In [2]:
import numpy as np
import psytrack as psy
import os
import datetime

%load_ext autoreload
%autoreload 2

import helperFns as mf
import fitFns as ff

import pickle 
import copy

curPath = os.path.abspath(os.getcwd())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
IDs = ('GS027', 'GS028', 'GS029', 'GS030', 'GS037', 'GS040', 'JC025', 'JC028', 'JC029', 'JC039', 'JC044', 'JC047', 'JC048', 'JC052', 'JC057', 'JC059', 'JC061', 'JC062', 'JC067')

useVel = False
saveDat = True

weights = {'bias': 1, 
           's_high': 1,
           's_low': 1}

K = np.sum([weights[i] for i in weights.keys()])

hyper= {'sigInit': 2**4.,
        'sigma': [2**-4.]*K,
        'sigDay': 2**-4.}

optList = ['sigma','sigDay']

for ID in IDs:

    print('Starting ' + ID)
    
    keyword = "testing"
    stc = 'testing'
    
    cutoff = 60
    numSessions = None
    accCutoff = 0.7
    tD = mf.getD(ID, keyword = keyword, cutoff = cutoff, accCutoff=accCutoff, sessionCutoff = numSessions, txt = None, useVel = useVel)
    
    tD2 = copy.deepcopy(tD)

    hyp, evd, wMode, hess_info = psy.hyperOpt(tD2, hyper, weights, optList)

    psytrack = {
        'wMode': wMode,
        'weights': weights,
        'hyp': hyp,
    }

    tD['psytrack'] = psytrack

    today = datetime.datetime.now()
    date_time = today.strftime("%y%m%d")
    tmp = 'new_data/' + date_time + '/'
   
    if not os.path.isdir(tmp):
        os.makedirs(tmp)

    if saveDat:
        saveName = tmp + "{}_biasTestingData.pkl".format(ID)
        with open(saveName, 'wb') as f:
            pickle.dump(tD, f)

    tD = mf.getD(ID, keyword = keyword, cutoff = cutoff, accCutoff=accCutoff, sessionCutoff = numSessions, removeProbes = False, txt = None, useVel = useVel)
    
    psychFit = ff.getPsychFit(tD, fitMethod = 'bads', nF = 2, stc = stc) # Use 'pymc' for sampling from posterior, 'bads' for point estimates
    
    if saveDat:
        saveName = tmp + "{}_biasTestingFitData.pkl".format(ID)
        with open(saveName, 'wb') as f:
            pickle.dump(psychFit, f)


Starting GS040
